<a href="https://colab.research.google.com/github/Arham-Reza/Stress_Detection_wesad/blob/main/Stress_Detection_S2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import seaborn as sns
# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler, LabelEncoder
# Classification Report
from sklearn.metrics import classification_report

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df=pd.read_csv('/content/drive/MyDrive/chest_data.csv')

In [6]:
df.head()

,Unnamed: 0,chest_ax,chest_ay,chest_az,chest_ecg,chest_emg,chest_eda,chest_temp,chest_resp,label
0,0,0.9554,-0.2220,-0.5580,0.021423,-0.004440,5.250549,30.120758,-1.148987,0.0
1,1,0.9258,-0.2216,-0.5538,0.020325,0.004349,5.267334,30.129517,-1.124573,0.0
2,2,0.9082,-0.2196,-0.5392,0.016525,0.005173,5.243301,30.138214,-1.152039,0.0
3,3,0.8974,-0.2102,-0.5122,0.016708,0.007187,5.249405,30.129517,-1.158142,0.0
4,4,0.8882,-0.2036,-0.4824,0.011673,-0.015152,5.286407,30.130951,-1.161194,0.0


In [7]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
df.shape

(4255300, 9)

In [9]:
print(df.label.value_counts())

0.0    2142701
1.0     800800
4.0     537599
2.0     430500
3.0     253400
6.0      45500
7.0      44800
Name: label, dtype: int64


In [10]:
df=df[(df.label != 0) & (df.label != 6) & (df.label !=7)]

In [11]:
df.shape

(2022299, 9)

In [12]:
print(df.label.value_counts())

1.0    800800
4.0    537599
2.0    430500
3.0    253400
Name: label, dtype: int64


In [14]:
label_encoder = LabelEncoder()

encoded_labels = label_encoder.fit_transform(df['label'])

np.unique(encoded_labels)

array([0, 1, 2, 3])

In [15]:
df['label'] = encoded_labels

In [16]:
df['label'].value_counts()

0    800800
3    537599
1    430500
2    253400
Name: label, dtype: int64

In [17]:
df.describe()

,chest_ax,chest_ay,chest_az,chest_ecg,chest_emg,chest_eda,chest_temp,chest_resp,label
count,2.022299e+06,2.022299e+06,2.022299e+06,2.022299e+06,2.022299e+06,2.022299e+06,2.022299e+06,2.022299e+06,2.022299e+06
mean,6.905893e-01,-1.254201e-01,-5.541978e-01,1.189813e-03,-2.410597e-03,1.477928e+00,3.056060e+01,5.047145e-02,1.260989e+00
std,1.585121e-01,5.674518e-02,2.676459e-01,1.364764e-01,9.685848e-03,9.621079e-01,1.592403e+00,2.229207e+00,1.231469e+00
min,3.116000e-01,-4.194000e-01,-1.004000e+00,-1.499405e+00,-2.458191e-01,2.632141e-01,2.804526e+01,-2.019653e+01,0.000000e+00
25%,6.046000e-01,-1.446000e-01,-7.346000e-01,-6.037903e-02,-7.553101e-03,7.904053e-01,2.856464e+01,-1.321411e+00,0.000000e+00
50%,6.286000e-01,-1.240000e-01,-7.094000e-01,1.199341e-02,-1.556396e-03,1.216888e+00,3.127814e+01,-1.724243e-01,1.000000e+00
75%,8.760000e-01,-7.340002e-02,-2.814000e-01,4.618835e-02,3.295898e-03,1.777267e+00,3.193958e+01,1.438904e+00,3.000000e+00
max,1.380600e+00,2.024000e-01,1.041800e+00,1.415726e+00,1.978912e-01,5.756760e+00,3.437039e+01,1.927032e+01,3.000000e+00


In [18]:
x = df.drop("label", axis = 1)
sc = StandardScaler()
x = pd.DataFrame(sc.fit_transform(x))
y = df["label"]

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [20]:
print("Shape of training dataset : ", x_train.shape)
print("Shape of testing dataset : " , x_test.shape)

Shape of training dataset :  (1617839, 8)
Shape of testing dataset :  (404460, 8)


In [21]:
model=tf.keras.Sequential([
                        tf.keras.layers.Dense(60, input_shape = (8,), activation = "relu"),
                        tf.keras.layers.Dense(15, activation = "relu"),
                        tf.keras.layers.Dropout(0.2),
                        tf.keras.layers.Dense(4, activation = "softmax")
                         
])

In [22]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])

In [23]:
model.fit(x_train, y_train, validation_split=0.1, verbose=1, epochs=10)

Epoch 1/10
45502/45502 [==============================] - 94s 2ms/step - loss: 0.0086 - accuracy: 0.9975 - val_loss: 1.3411e-04 - val_accuracy: 1.0000
Epoch 2/10
45502/45502 [==============================] - 90s 2ms/step - loss: 0.0021 - accuracy: 0.9993 - val_loss: 7.3635e-05 - val_accuracy: 1.0000
Epoch 3/10
45502/45502 [==============================] - 93s 2ms/step - loss: 0.0014 - accuracy: 0.9995 - val_loss: 2.7740e-05 - val_accuracy: 1.0000
Epoch 4/10
45502/45502 [==============================] - 88s 2ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 2.8936e-04 - val_accuracy: 1.0000
Epoch 5/10
45502/45502 [==============================] - 95s 2ms/step - loss: 0.0011 - accuracy: 0.9997 - val_loss: 1.2490e-04 - val_accuracy: 1.0000
Epoch 6/10
45502/45502 [==============================] - 93s 2ms/step - loss: 0.0011 - accuracy: 0.9997 - val_loss: 2.6905e-04 - val_accuracy: 1.0000
Epoch 7/10
45502/45502 [==============================] - 89s 2ms/step - loss: 0.0010 - accura

In [24]:
evaluation_results = model.evaluate(x_test, y_test)

print(f'Performance on test set\nLoss: {evaluation_results[0]}\tAccuaracy: {evaluation_results[1]}')

12640/12640 [==============================] - 18s 1ms/step - loss: 2.1457e-04 - accuracy: 1.0000
Performance on test set
Loss: 0.000214570842217654	Accuaracy: 0.9999629259109497


In [25]:
test_pred = model.predict(x_test)
test_pred.shape

(404460, 4)

In [26]:
test_pred

array([[1.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
       [1.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
       [3.330241e-08, 0.000000e+00, 1.000000e+00, 0.000000e+00],
       ...,
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 1.000000e+00],
       [1.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
       [1.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00]],
      dtype=float32)

In [27]:
y_pred = np.argmax(test_pred, axis = 1)
y_pred.shape

(404460,)

In [28]:
y_pred

array([0, 0, 2, ..., 3, 0, 0])

In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    160051
           1       1.00      1.00      1.00     85958
           2       1.00      1.00      1.00     50748
           3       1.00      1.00      1.00    107703

    accuracy                           1.00    404460
   macro avg       1.00      1.00      1.00    404460
weighted avg       1.00      1.00      1.00    404460

